# 00 Stationarity and Unit Roots

ADF/KPSS, differencing, and spurious regression intuition.


## Table of Contents
- [Load macro series](#load-macro-series)
- [Transformations](#transformations)
- [ADF/KPSS tests](#adf-kpss-tests)
- [Spurious regression demo](#spurious-regression-demo)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Time-series econometrics notebooks build the classical toolkit you need before trusting macro regressions:
- stationarity + unit roots,
- cointegration + error correction,
- VAR dynamics and impulse responses.


## Prerequisites (Quick Self-Check)
- Completed Part 01 macro panel notebooks (or have `panel_monthly.csv` / sample available).
- Comfort with differencing/log transforms and reading time series plots.

## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Running tests without plotting or transforming the series first.
- Treating impulse responses as structural causality without an identification story.

## Quick Fixes (When You Get Stuck)
- If you see `ModuleNotFoundError`, re-run the bootstrap cell and restart the kernel; make sure `PROJECT_ROOT` is the repo root.
- If a `data/processed/*` file is missing, either run the matching build script (see guide) or use the notebook’s `data/sample/*` fallback.
- If results look “too good,” suspect leakage; re-check shifts, rolling windows, and time splits.
- If a model errors, check dtypes (`astype(float)`) and missingness (`dropna()` on required columns).

## Matching Guide
- `docs/guides/07_time_series_econ/00_stationarity_unit_roots.md`



## How To Use This Notebook
- Work section-by-section; don’t skip the markdown.
- Most code cells are incomplete on purpose: replace TODOs and `...`, then run.
- After each section, write 2–4 sentences answering the interpretation prompts (what changed, why it matters).
- Prefer `data/processed/*` if you have built the real datasets; otherwise use the bundled `data/sample/*` fallbacks.
- Use the **Checkpoint (Self-Check)** section to catch mistakes early.
- Use **Solutions (Reference)** only to unblock yourself; then re-implement without looking.
- Use the matching guide (`docs/guides/07_time_series_econ/00_stationarity_unit_roots.md`) for the math, assumptions, and deeper context.



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Learn the stationarity toolkit that prevents common macro mistakes:
- spurious regression
- over-trusting p-values on trending series
- misinterpreting dynamics



## Primer: pandas time series essentials (indexing, resampling, lags)

Most “mysterious bugs” in time series work come from index and alignment mistakes. This primer gives you the minimum patterns to avoid them.

### 1) DatetimeIndex (the first thing to verify)

Most time-series operations assume a `DatetimeIndex`:

```python
import pandas as pd

df = df.copy()
df.index = pd.to_datetime(df.index)
df = df.sort_index()
assert isinstance(df.index, pd.DatetimeIndex)
```

**Expected output / sanity checks**
- `df.index.min(), df.index.max()` look reasonable
- `df.index.is_monotonic_increasing` is `True`

### 2) Resampling (frequency alignment)

Resampling converts one frequency to another. Choose the aggregation rule intentionally.

```python
# month-end last value (end-of-period)
df_me_last = df.resample("ME").last()

# month-end mean (average-of-period)
df_me_mean = df.resample("ME").mean()

# quarter-end mean
df_q_mean = df.resample("QE").mean()
```

**Interpretation matters**
- `.last()` treats end-of-period value as “the period’s value.”
- `.mean()` treats the period average as “the period’s value.”

### 3) Alignment and merging

When joining series, always check missingness after the join:

```python
merged = df1.join(df2, how="outer").sort_index()
print(merged.isna().sum().sort_values(ascending=False).head(10))
```

### 4) Lags and rolling windows (watch for leakage!)

```python
# lag 1 period (past-only)
df["x_lag1"] = df["x"].shift(1)

# rolling mean using past values ending at t
df["x_roll12"] = df["x"].rolling(12).mean()
```

**Leakage pitfalls**
- `shift(-1)` uses the future.
- `rolling(..., center=True)` uses the future.

### 5) A quick workflow you should repeat

1) Set and verify DatetimeIndex.
2) Resample intentionally (mean vs last).
3) Join and inspect missingness.
4) Add lags/rolls (past-only).
5) `dropna()` to build a clean modeling table.


## Primer: Classical time-series econometrics with `statsmodels` (ADF/KPSS, cointegration, VAR/IRF)

This repo already teaches time-aware evaluation for ML. This primer introduces the classical econometrics toolkit for time series:
- stationarity / unit roots,
- cointegration and error correction ideas,
- VARs and impulse responses.

Deep theory is in the guides; this primer focuses on “how to use the tools correctly.”

### Before you start: what you should always do

1) **Plot the series in levels** (look for trends, breaks).
2) **Choose transformations** (diff/logdiff) for stationarity.
3) **Drop missing values** before tests/models.

### Stationarity tests (ADF / KPSS)

Two common tests:
- **ADF**: null = unit root (nonstationary)
- **KPSS**: null = stationary

```python
from statsmodels.tsa.stattools import adfuller, kpss

x = df["SERIES"].dropna().to_numpy()

adf_stat, adf_p, *_ = adfuller(x)
kpss_stat, kpss_p, *_ = kpss(x, regression="c", nlags="auto")

print("ADF p:", adf_p, "KPSS p:", kpss_p)
```

**Expected output / sanity check**
- trending level series often: ADF p not small, KPSS p small
- differenced series often: ADF p small, KPSS p not small

### Cointegration (Engle–Granger test)

If two series are individually nonstationary but move together long-run, they may be cointegrated.

```python
from statsmodels.tsa.stattools import coint

y = df["Y"].dropna()
x = df["X"].dropna()

stat, p, _ = coint(y, x)
print("coint p:", p)
```

### VAR (vector autoregression)

VAR models multiple stationary-ish series jointly.

```python
from statsmodels.tsa.api import VAR

X = df[["UNRATE", "FEDFUNDS", "INDPRO"]].astype(float).dropna()
X = X.diff().dropna()  # common stationarity transform

model = VAR(X)
res = model.fit(maxlags=8, ic="aic")  # or choose lags manually
print("lags chosen:", res.k_ar)
print(res.summary())
```

**Expected output / sanity check**
- `res.k_ar` is the chosen lag length
- `res.is_stable(verbose=False)` should be True for a stable VAR

### Granger causality (predictive, not causal)

```python
res.test_causality("UNRATE", ["FEDFUNDS"]).summary()
```

Interpretation: “do lagged FEDFUNDS help predict UNRATE beyond lagged UNRATE?”

### Impulse responses (IRFs)

```python
irf = res.irf(12)
irf.plot(orth=True)  # orthogonalized IRFs (ordering matters)
```

**Important:** orthogonalized IRFs depend on a Cholesky ordering.

### Common pitfalls (and quick fixes)

- **Nonstationary inputs:** VAR on levels can be nonsense.
  - Fix: difference/logdiff; or use cointegration/VECM logic.
- **Too many lags:** eats degrees of freedom and can destabilize the model.
  - Fix: try smaller maxlags, compare AIC/BIC, check diagnostics.
- **Misinterpreting Granger causality:** it is about predictive content, not structural causality.
- **Forgetting ordering:** orth IRFs change when you reorder variables.


<a id="load-macro-series"></a>
## Load macro series

### Background
Stationarity analysis is only meaningful if your time index is correct.
So the first task is: load a clean monthly panel with a proper `DatetimeIndex`.

### What you should see
- a DataFrame indexed by dates (monthly).
- key macro columns like CPI, unemployment, production.

### Interpretation prompts
- Which of these series looks trending in levels?
- Which series might be closer to stationary already (in levels)?

### Goal
Load the macro monthly panel.



### Your Turn: Load panel_monthly.csv (or sample)


In [ ]:
import pandas as pd

path = PROCESSED_DIR / 'panel_monthly.csv'
if path.exists():
    df = pd.read_csv(path, index_col=0, parse_dates=True)
else:
    df = pd.read_csv(SAMPLE_DIR / 'panel_monthly_sample.csv', index_col=0, parse_dates=True)

df = df.dropna().copy()
df.head()



<a id="transformations"></a>
## Transformations

### Background
Many macro series are nonstationary in levels.
Common fixes are:
- differences (change),
- percent changes (growth rates),
- log-differences (approx growth rates for positive series).

### What you should see
- transformed columns with fewer trends.
- a smaller DataFrame after `dropna()` (because differencing creates missing first row).

### Interpretation prompts
- Why might log-differences be preferred for production indexes?
- What information do you lose when you difference?

### Goal
Create stationary-ish transformations (diff, pct change, log diff).



### Your Turn: Differences and growth rates


In [ ]:
import numpy as np

# TODO: Pick a few series and create transformations
tmp = df[['CPIAUCSL', 'UNRATE', 'INDPRO']].astype(float).copy()
tmp['dCPI'] = tmp['CPIAUCSL'].diff()
tmp['dUNRATE'] = tmp['UNRATE'].diff()

# log-diff for industrial production (example)
x = tmp['INDPRO'].where(tmp['INDPRO'] > 0)
tmp['dlog_INDPRO'] = np.log(x).diff()

tmp = tmp.dropna()
tmp.head()



<a id="adf-kpss-tests"></a>
## ADF/KPSS tests

### Background
ADF and KPSS are complementary diagnostics:
- ADF null: unit root (nonstationary)
- KPSS null: stationary

No single p-value is a proof. Use tests alongside plots and economic context.

### What you should see
- different p-values for level vs differenced series.
- clearer stationarity evidence after transformation.

### Interpretation prompts
- In words: what does a small ADF p-value suggest?
- In words: what does a small KPSS p-value suggest?

### Goal
Run stationarity diagnostics on levels vs transformed series.



### Your Turn: ADF and KPSS


In [ ]:
from statsmodels.tsa.stattools import adfuller, kpss

# TODO: Choose one series and compare levels vs diff
x = df['CPIAUCSL'].astype(float).dropna()
dx = x.diff().dropna()

out = {
    'adf_p_level': adfuller(x)[1],
    'adf_p_diff': adfuller(dx)[1],
    'kpss_p_level': kpss(x, regression='c', nlags='auto')[1],
    'kpss_p_diff': kpss(dx, regression='c', nlags='auto')[1],
}
out



<a id="spurious-regression-demo"></a>
## Spurious regression demo

### Background
A classic macro trap is regressing one trending series on another.
You can get a high $R^2$ and significant coefficients even when the relationship is meaningless.

### What you should see
- the levels regression often has a higher $R^2$ than the differences regression.
- this demonstrates why stationarity checks are a prerequisite for inference.

### Interpretation prompts
- Why can $R^2$ be high in a spurious regression?
- What would you do next if you *needed* a meaningful long-run relationship? (hint: cointegration)

### Goal
Show how levels-on-levels regressions can look good for the wrong reasons.



### Your Turn: Levels vs differences


In [ ]:
import statsmodels.api as sm

tmp2 = df[['CPIAUCSL', 'INDPRO']].astype(float).dropna()

# Levels regression
res_lvl = sm.OLS(tmp2['CPIAUCSL'], sm.add_constant(tmp2[['INDPRO']], has_constant='add')).fit()

# Differences regression
d = tmp2.diff().dropna()
res_diff = sm.OLS(d['CPIAUCSL'], sm.add_constant(d[['INDPRO']], has_constant='add')).fit()

print('R2 levels:', res_lvl.rsquared)
print('R2 diffs :', res_diff.rsquared)



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
import pandas as pd

# TODO: Validate your time series table is well-formed.
# Example (adjust variable names):
# assert isinstance(df.index, pd.DatetimeIndex)
# assert df.index.is_monotonic_increasing
# assert df.shape[0] > 30
#
# TODO: If you built transformed series (diff/logdiff), confirm no future leakage.
# Hint: transformations should only use past/current values (shift/diff), never future.
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Load macro series</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_stationarity_unit_roots — Load macro series
import pandas as pd

path = PROCESSED_DIR / 'panel_monthly.csv'
if path.exists():
    df = pd.read_csv(path, index_col=0, parse_dates=True)
else:
    df = pd.read_csv(SAMPLE_DIR / 'panel_monthly_sample.csv', index_col=0, parse_dates=True)

df = df.dropna().copy()
df.head()
```

</details>

<details><summary>Solution: Transformations</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_stationarity_unit_roots — Transformations
# Example: difference CPI and unemployment
df_t = df[['CPIAUCSL', 'UNRATE']].astype(float).copy()
df_t['dCPI'] = df_t['CPIAUCSL'].diff()
df_t['dUNRATE'] = df_t['UNRATE'].diff()
df_t = df_t.dropna()
df_t.head()
```

</details>

<details><summary>Solution: ADF/KPSS tests</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_stationarity_unit_roots — ADF/KPSS tests
from statsmodels.tsa.stattools import adfuller, kpss

x = df['CPIAUCSL'].astype(float).dropna()
dx = x.diff().dropna()

adf_p_level = adfuller(x)[1]
adf_p_diff = adfuller(dx)[1]
kpss_p_level = kpss(x, regression='c', nlags='auto')[1]
kpss_p_diff = kpss(dx, regression='c', nlags='auto')[1]

{'adf_p_level': adf_p_level, 'adf_p_diff': adf_p_diff, 'kpss_p_level': kpss_p_level, 'kpss_p_diff': kpss_p_diff}
```

</details>

<details><summary>Solution: Spurious regression demo</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_stationarity_unit_roots — Spurious regression demo
import statsmodels.api as sm

# Levels-on-levels can look 'significant' even when dynamics are mis-specified.
tmp = df[['CPIAUCSL', 'INDPRO']].astype(float).dropna()
res_lvl = sm.OLS(tmp['CPIAUCSL'], sm.add_constant(tmp[['INDPRO']], has_constant='add')).fit()
res_diff = sm.OLS(tmp['CPIAUCSL'].diff().dropna(), sm.add_constant(tmp['INDPRO'].diff().dropna(), has_constant='add')).fit()

(res_lvl.rsquared, res_diff.rsquared)
```

</details>

